# Training a Neural Causal Model

As Robert Osazuwa Ness details in his book [causal ai](https://www.manning.com/books/causal-ai):

> We regularly work with functions in causal modeling and inference, and sometimes it makes sense to approximate them, *so long as the approximations preserve the causal information we care about.*.. [i]n this section, we’ll do this mapping between a node and its parents with the variational autoencoder (VAE) framework. We’ll train two deep neural nets in the VAE, one of which *maps parent cause variables to a distribution of the outcome variable*, and another that *maps the outcome variable to a distribution of the cause variables.*

We will use this framework to build a neural causal model of an image using the combined dataset of [MNIST](https://en.wikipedia.org/wiki/MNIST_database) and [TypefaceMNIST](https://paperswithcode.com/dataset/typography-mnist) in which a binary label represents whether the digit is handwritten(i.e., 1) or typed (i.e., 0).

In [ ]:
!pip install "pyro-ppl"
!pip install "torch"
!pip install "torchvision"
!pip install "numpy==1.26.4"

## 1. Downloading and Preprocessing Data

In [ ]:
import torch
USE_CUDA = True
DEVICE_TYPE = torch.device("cuda" if USE_CUDA else "cpu")

In [ ]:
# class to combine the datasets
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
from torchvision import transforms

class CombinedDataset(Dataset):
  def __init__(self, csv_file):
    self.dataset = pd.read_csv(csv_file)

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    images = self.dataset.iloc[idx, 3:]
    images = np.array(images, dtype="float32")/255
    images = images.reshape(28, 28)
    transform = transforms.ToTensor()
    images = transform(images)
    digits = self.dataset.iloc[idx, 2]
    digits = np.array([digits], dtype="int")
    is_handwritten = self.dataset.iloc[idx, 1]
    is_handwritten = np.array([is_handwritten], dtype='float32')
    return images, digits, is_handwritten


In [ ]:
# download, split, load data
from torch.utils.data import DataLoader
from torch.utils.data import random_split
def setup_dataloaders(batch_size=64, use_cuda=USE_CUDA):
  combined_dataset = CombinedDataset(
      csv_file = "https://raw.githubusercontent.com/altdeep/causalML/master/datasets/combined_mnist_tmnist_data.csv"
  )
  kwargs = {'num_workers': 1, 'pin_memory': use_cuda}
  n = len(combined_dataset)
  train_size = int(0.8 *n)
  test_size = n-train_size
  train_dataset, test_dataset = random_split(
        combined_dataset,
        [train_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )
  train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        **kwargs
    )
  test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=True,
        **kwargs
    )
  return train_loader, test_loader

## 2. Creating the Encoder and Decoder

Similar to the traditional [Variational AutoEncoder (VAE)](https://en.wikipedia.org/wiki/Variational_autoencoder) used in machine learning the causal form is also composed of an encoder and decoder, but is used to model the *causal DAG of generating the image* (i.e.,the DGP that generates the MNST image). To that end we model the causal drivers of what determines the image we see. For this example, the main drivers of the MNST images are  1) the label/type of *digit* 2) the label of is it *handwritten* and 3) (and in my opinion the most important) the latent causes as represented by the variable Z. As Robert Osazuwa Ness details in his book [causal ai](https://www.manning.com/books/causal-ai):

>  Z appears as a new parent in the causal DAG...we view *digit* and *is-handwritten* as causal drivers of what we see in the image. Yet there are other elements of the image (e.g., the stroke thickness of a handwritten character, or the font of a typed character) that are also causes of what we see in the image. *We’ll think of Z as a continuous latent stand-in for all of these other causes of the image that we are not explicitly modeling...it is important to remember that the representation we learn for Z is a stand-in for latent causes and is not the same as learning the actual latent causes.*

Accordingly, the encoder's job is to encode the explicit variables of image, the label of *is-handwritten* and the *digit* label into a latent variable Z which when combined with the explicit variables and fed into the decoder generates the image we see.


In [ ]:
# creating the Decoder
from torch import nn

class Decoder(nn.Module):
    def __init__(self, z_dim, hidden_dim):
        super().__init__()
        img_dim = 28 * 28
        digit_dim = 10
        is_handwritten_dim = 1
        self.softplus = nn.Softplus()
        self.sigmoid = nn.Sigmoid()
        encoding_dim = z_dim + digit_dim + is_handwritten_dim
        self.fc1 = nn.Linear(encoding_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, img_dim)

    def forward(self, z, digit, is_handwritten):
        input = torch.cat([z, digit, is_handwritten], dim=1)
        hidden = self.softplus(self.fc1(input))
        img_param = self.sigmoid(self.fc2(hidden))
        return img_param

# creating the Encoder
class Encoder(nn.Module):
    def __init__(self, z_dim, hidden_dim):
        super().__init__()
        img_dim = 28 * 28
        digit_dim = 10
        is_handwritten_dim = 1
        self.softplus = nn.Softplus()
        input_dim = img_dim + digit_dim + is_handwritten_dim
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, z_dim)
        self.fc22 = nn.Linear(hidden_dim, z_dim)

    def forward(self, img, digit, is_handwritten):
        input = torch.cat([img, digit, is_handwritten], dim=1)
        hidden = self.softplus(self.fc1(input))
        z_loc = self.fc21(hidden)
        z_scale = torch.exp(self.fc22(hidden))
        return z_loc, z_scale

## 3. Pyro Functions used to model the Causal Variational AutoEncoder(VAE)

As detailed above the decoder represents modeling the causal [Markov kernel](https://math.stackexchange.com/questions/4594349/question-about-the-definition-of-markov-kernel); that is the conditional probability of $P(X | digit, is-handwritten, Z)$ where $X$ represents the image.  The latent variable Z, the label *digit* and the label *is-handwritten* are sampled from standard canonical distributions. In this case Z is sampled from a Normal distribution, the label *digit* is sampled from a Categorical distribution, and the label *is-handwritten* is sampled from a Bernoulli distribution.

Additionally as the book details since we are modeling the joint probability distribution (i.e. $P(X, digit, is-handwritten, Z)$), the random variable X which represents the pixel values of the image needs to be accounted for too. Even though the pixel values are not technically binary outcomes, since they can take values from 0 to 255, we model them using the Bernoulli distribution due to ease of use. This helper method is named **model** in the VAE.

To apply the model to the images in the training data the helper method **training_model** is used to condition the model on the data (i.e., the labels and images). Furthermore, we want to get good samples for the latent variable Z *(since it represents the image)* we implement a guide function that uses the encoder's output and canonical distributions to sample good values for Z. This helper method is named **training_guide** in the VAE.




In [ ]:
# Creating VAE
import pyro
import pyro.distributions as dist

dist.enable_validation(False)
class VAE(nn.Module):
    def __init__(
        self,
        z_dim=50,
        hidden_dim=400,
        use_cuda=USE_CUDA,
    ):
        super().__init__()
        self.use_cuda = use_cuda
        self.z_dim = z_dim
        self.hidden_dim = hidden_dim
        self.setup_networks()

    def setup_networks(self):
        self.encoder = Encoder(self.z_dim, self.hidden_dim)
        self.decoder = Decoder(self.z_dim, self.hidden_dim)
        if self.use_cuda:
            self.cuda()

    def model(self, data_size=1):
      pyro.module("decoder", self.decoder)
      options = dict(dtype=torch.float32, device=DEVICE_TYPE)
      z_loc = torch.zeros(data_size, self.z_dim, **options)
      z_scale = torch.ones(data_size, self.z_dim, **options)
      z = pyro.sample("Z", dist.Normal(z_loc, z_scale).to_event(1))
      p_digit = torch.ones(data_size, 10, **options)/10
      digit = pyro.sample(
          "digit",
          dist.OneHotCategorical(p_digit)
      )
      p_is_handwritten = torch.ones(data_size, 1, **options)/2
      is_handwritten = pyro.sample(
          "is_handwritten",
          dist.Bernoulli(p_is_handwritten).to_event(1)
      )
      img_param = self.decoder(z, digit, is_handwritten)
      img = pyro.sample("img", dist.Bernoulli(img_param).to_event(1))
      return img, digit, is_handwritten

    def training_model(self, img, digit, is_handwritten, batch_size):
      conditioned_on_data = pyro.condition(
          self.model,
          data={
            "digit": digit,
            "is_handwritten": is_handwritten,
            "img": img
          }
      )
      with pyro.plate("data", batch_size):
          img, digit, is_handwritten = conditioned_on_data(batch_size)
      return img, digit, is_handwritten

    def training_guide(self, img, digit, is_handwritten, batch_size):
      pyro.module("encoder", self.encoder)
      options = dict(dtype=torch.float32, device=DEVICE_TYPE)
      with pyro.plate("data", batch_size):
        z_loc, z_scale = self.encoder(img, digit, is_handwritten)
        normal_dist = dist.Normal(z_loc, z_scale).to_event(1)
        z = pyro.sample("Z", normal_dist)

# 4. Training the Causal Variational Autoencoder(VAE)

In addition to the traditional method of training VAEs i.e., minimizing the reconstruction error between the original and reconstructed/generated images, we also introduce a probabilistic approach to train the causal VAE. As Robert Osazuwa Ness details in his book [causal ai](https://www.manning.com/books/causal-ai):

> In practice, solely minimizing reconstruction error leads to overfitting and other issues, so we’ll opt for a probabilistic approach: *given an image, we’ll use our guide function to sample a value of Z from $P(Z|image, is-handwritten, digit).$* [And] [t]hen we’ll plug that value into our model’s decoder....

Unfortunately, we cannot directly sample from the conditional probability distribution of $P(Z|image, is-handwritten, digit)$; in other words it is intractable. The solution to this problem is to *approximate the conditional probability distribution by using tractable distributions;* this training procedure is known as [variational inference](https://ermongroup.github.io/cs228-notes/inference/variational/)

We will use the previous guide function i.e., **training_guide** to approximate the conditional probability distribution of $P(Z|image, is-handwritten, digit)$ (which in the literature is represented by the letter Q/q). We denote the distribution that we are approximating as $Q(Z | X, is-handwritten, digit)$ which represents the [variational distribution](https://ai.stackexchange.com/questions/41834/what-exactly-is-meant-by-variational-distribution). Interestingly, *changing the weights of the encoder represents shifting the variational distribution.* As Robert Osazuwa Ness details in his book [causal ai](https://www.manning.com/books/causal-ai):

> A change in the weights of the encoder represents a shifting of the variational distribution. Training will optimize the weights of the encoder such that the variational distribution shifts toward $P(Z|image, is-handwritten, digit).$

Training through [variational inference](https://ermongroup.github.io/cs228-notes/inference/variational/) works by minimizing the [Kullback Leibler (KL) divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence) between the two distributions and minimizing the negative [ELBO loss](https://en.wikipedia.org/wiki/Evidence_lower_bound). Doing so will indirectly minimize the KL divergence between $Q(Z | X, is-handwritten, digit)$ and $P(Z|image, is-handwritten, digit)$.




In [ ]:
import matplotlib.pyplot as plt
import torch.nn.functional as F

# helper function plotting images
def plot_image(img, title=None):     #1
    fig = plt.figure()
    plt.imshow(img.cpu(), cmap='Greys_r', interpolation='nearest')
    if title is not None:
        plt.title(title)
    plt.show()

# helper function for reconstruct and viewing
def reconstruct_img(vae, img, digit, is_hw, use_cuda=USE_CUDA):     #1
    img = img.reshape(-1, 28 * 28)
    digit = F.one_hot(torch.tensor(digit), 10)
    is_hw = torch.tensor(is_hw).unsqueeze(0)
    if use_cuda:
        img = img.cuda()
        digit = digit.cuda()
        is_hw = is_hw.cuda()
    z_loc, z_scale = vae.encoder(img, digit, is_hw)
    z = dist.Normal(z_loc, z_scale).sample()
    img_expectation = vae.decoder(z, digit, is_hw)
    return img_expectation.squeeze().view(28, 28).detach()

def compare_images(img1, img2):    #2
    fig = plt.figure()
    ax0 = fig.add_subplot(121)
    plt.imshow(img1.cpu(), cmap='Greys_r', interpolation='nearest')
    plt.axis('off')
    plt.title('original')
    ax1 = fig.add_subplot(122)
    plt.imshow(img2.cpu(), cmap='Greys_r', interpolation='nearest')
    plt.axis('off')
    plt.title('reconstruction')
    plt.show()

# helper data processing functions for training
 def get_random_example(loader):    #1
    random_idx = np.random.randint(0, len(loader.dataset))    #1
    img, digit, is_handwritten = loader.dataset[random_idx]     #1
    return img.squeeze(), digit, is_handwritten    #1

def reshape_data(img, digit, is_handwritten):     #2
    digit = F.one_hot(digit, 10).squeeze()     #2
    img = img.reshape(-1, 28*28)     #2
    return img, digit, is_handwritten     #2

def generate_coded_data(vae, use_cuda=USE_CUDA):     #3
    z_loc = torch.zeros(1, vae.z_dim)     #3
    z_scale = torch.ones(1, vae.z_dim)     #3
    z = dist.Normal(z_loc, z_scale).to_event(1).sample()     #3
    p_digit = torch.ones(1, 10)/10     #3
    digit = dist.OneHotCategorical(p_digit).sample()     #3
    p_is_handwritten = torch.ones(1, 1)/2     #3
    is_handwritten = dist.Bernoulli(p_is_handwritten).sample()    #3
    if use_cuda:     #3
        z = z.cuda() #3
        digit = digit.cuda() #3
        is_handwritten = is_handwritten.cuda()     #3
    img = vae.decoder(z, digit, is_handwritten)     #3
    return img, digit, is_handwritten    #3

def generate_data(vae, use_cuda=USE_CUDA):     #4
    img, digit, is_handwritten = generate_coded_data(vae, use_cuda)    #4
    img = img.squeeze().view(28, 28).detach()    #4
    digit = torch.argmax(digit, 1)   #4
    is_handwritten = torch.argmax(is_handwritten, 1)     #4
    return img, digit, is_handwritten     #4

In [ ]:
# setting up training
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam

pyro.clear_param_store()     #1
vae = VAE()    #2
train_loader, test_loader = setup_dataloaders(batch_size=256)    #3
svi_adam = Adam({"lr": 1.0e-3})     #4
model = vae.training_model    #5
guide = vae.training_guide    #5
svi = SVI(model, guide, svi_adam, loss=Trace_ELBO())     #5

# setting up evaluation tester
def test_epoch(vae, test_loader):
    epoch_loss_test = 0     #1
    for img, digit, is_hw in test_loader:    #1
        batch_size = img.shape[0]    #1
        if USE_CUDA:    #1
            img = img.cuda()    #1
            digit = digit.cuda()     #1
            is_hw = is_hw.cuda()  #1
        img, digit, is_hw = reshape_data(     #1
            img, digit, is_hw     #1
        )   #1
        epoch_loss_test += svi.evaluate_loss(     #1
            img, digit, is_hw, batch_size    #1
        )    #1
    test_size = len(test_loader.dataset)    #1
    avg_loss = epoch_loss_test/test_size     #1
    print("Epoch: {} avg. test loss: {}".format(epoch, avg_loss))     #1
    print("Comparing a random test image to its reconstruction:")    #2
    random_example = get_random_example(test_loader)     #2
    img_r, digit_r, is_hw_r = random_example    #2
    img_recon = reconstruct_img(vae, img_r, digit_r, is_hw_r)     #2
    compare_images(img_r, img_recon)     #2
    print("Generate a random image from the model:")    #3
    img_gen, digit_gen, is_hw_gen = generate_data(vae)    #3
    plot_image(img_gen, "Generated Image")     #3
    print("Intended digit: ", int(digit_gen))    #3
    print("Intended as handwritten: ", bool(is_hw_gen == 1))     #3

In [ ]:
# the training and evaluation loop

NUM_EPOCHS = 2500
TEST_FREQUENCY = 10

train_loss = []
train_size = len(train_loader.dataset)

for epoch in range(0, NUM_EPOCHS+1):   #1
    loss = 0
    for img, digit, is_handwritten in train_loader:
        batch_size = img.shape[0]
        if USE_CUDA:
            img = img.cuda()
            digit = digit.cuda()
            is_handwritten = is_handwritten.cuda()
        img, digit, is_handwritten = reshape_data(
            img, digit, is_handwritten
        )
        loss += svi.step(    #2
            img, digit, is_handwritten, batch_size     #2
        )     #2
    avg_loss = loss / train_size
    print("Epoch: {} avgs training loss: {}".format(epoch, loss))
    train_loss.append(avg_loss)
    if epoch % TEST_FREQUENCY == 0:    #3
        test_epoch(vae, test_loader)    #3
